In [3]:
!pip install mxnet
!pip install gluonnlp
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 21.4 MB/s eta 0:00:00










  Attempting uninstall: graphviz

    Found existing installation: graphviz 0.20.1

    Uninstalling graphviz-0.20.1:

      Successfully uninstalled graphviz-0.20.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 13.2 MB/s eta 0:00:00

  Preparing metadata (setup.py) ... done






  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=681084 sha256=8d6dc3e69c27f8ea829e59d415b47c96b709723dd959ab5e330b2e93878775f0

  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3

Successfully built gluonnlp



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



In [1]:
import pandas as pd
import numpy as np
import os
import gc

import random

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# seed 값 설정
seed = 555
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import transformers
from transformers import AdamW

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
MODEL_TYPE = 'xlm-roberta-base'
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_TYPE)

In [4]:
# XLM-RoBERTa vocab크기 확인
tokenizer.vocab_size

250002

In [5]:
# XLM-RoBERTa vocab 확인
list(tokenizer.get_vocab())[:10]

['<s>', '<pad>', '</s>', '<unk>', ',', '.', '▁', 's', '▁de', '-']

In [6]:
device= torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [7]:
path = '/kaggle/input/jbnu-swuniv-ai/'
train = pd.read_csv(path+"/pre_train.csv")
test = pd.read_csv(path+"/pre_test.csv")

In [8]:
label_mapping = {label: i for i, label in enumerate(train["label"].unique())}
num_labels = len(label_mapping)

In [9]:
from sklearn.model_selection import train_test_split
train_dataset, val_dataset = train_test_split(train, test_size = 0.1, random_state=seed)

In [10]:
train_dataset.index=[i for i in range(len(train_dataset))]
val_dataset.index=[i for i in range(len(val_dataset))]
val_dataset

,Unnamed: 0,id,Filename,Title,prompt,label,label_num
0,1068,1068,0658001728.jpg,The Food CombiningBlood Type Diet Solution A P...,food combining blood type diet solution half b...,"Health, Fitness, Dieting",4
1,36785,36785,0071848886.jpg,CompTIA Network Certification Study Guide Sixt...,book cover exam guide network neon virtual net...,"Computers, Technology",7
2,15319,15319,0745670369.jpg,Sex Addiction A Critical History,sex addiction critical history book cover pape...,"Health, Fitness, Dieting",4
3,16192,16192,0892723718.jpg,Cooking Down East,book cover cooking east mardie standish paperb...,"Cookbooks, Food, Wine",0
4,53983,53983,1476710406.jpg,La distancia entre nosotros Atria Espanol Span...,close person walking dirt road incoherents emu...,Law,21
...,...,...,...,...,...,...,...
6864,64417,64417,1566565065.jpg,A Travellers History of New Zealand and the So...,new zealand south pacific islands illustration...,"Biographies, Memoirs",22
6865,25834,25834,0618800425.jpg,Curious George at the Zoo A Touch and Feel Book,close book panda monkey various animals zoo ge...,Childrens Books,12
6866,62165,62165,1452114307.jpg,Tartine Book No 3 Modern Ancient Classic Whole,picture book cover variety different foods gas...,"Cookbooks, Food, Wine",0
6867,42511,42511,0425257835.jpg,Voices of the Pacific Untold Stories from the ...,voices pacific adam makos paperback cover book...,"Biographies, Memoirs",22


In [11]:
MAX_LEN = 400

In [12]:
from torch.utils.data import Dataset, DataLoader

# train, val에 사용
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        # 데이터프레임 칼럼 들고오기
        sentence1 = self.df_data.loc[index, 'Title']
        sentence2 = self.df_data.loc[index, 'prompt']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    truncation=True,
                    return_attention_mask = True,   
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        # 숫자로 변환된 label을 텐서로 변환
        target = torch.tensor(self.df_data.loc[index, 'label_num'])
        # input_ids, attention_mask, label을 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask, target)

        return sample

    def __len__(self):
        return len(self.df_data)

In [13]:
# test 예측에 사용
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):

        sentence1 = self.df_data.loc[index, 'Title']
        sentence2 = self.df_data.loc[index, 'prompt']


        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,           
                    add_special_tokens = True,      
                    max_length = MAX_LEN,           
                    pad_to_max_length = True,
                    return_attention_mask = True,   
                    truncation=True,
                    return_tensors = 'pt',          
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        # input_ids, attention_mask를 하나의 인풋으로 묶음
        sample = (padded_token_list, att_mask)

        return sample


    def __len__(self):
        return len(self.df_data)

In [14]:
L_RATE = 1e-5

BATCH_SIZE = 16 # batch size가 클수록 global minimum에 도달하는 속도가 증가합니다. (GPU 메모리에 따라 변경해 주세요, 너무 크면 OOM 문제가 발생합니다.)
NUM_CORES = os.cpu_count() # Dataloader에 사용됩니다.

NUM_CORES

2

In [15]:
from transformers import XLMRobertaForSequenceClassification

model = XLMRobertaForSequenceClassification.from_pretrained(
    MODEL_TYPE,
    num_labels = 24,
)

model.to(device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.d

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [16]:
optimizer = AdamW(model.parameters(),
              lr = L_RATE,
              eps = 1e-8
            )

In [17]:
train_data = CompDataset(train_dataset)
val_data = CompDataset(val_dataset)
test_data = TestDataset(test)

train_dataloader = DataLoader(train_data,
                                batch_size=BATCH_SIZE,
                                shuffle=True,
                                num_workers=0)

val_dataloader = DataLoader(val_data,
                            batch_size=BATCH_SIZE,
                            shuffle=True,
                            num_workers=0)

test_dataloader = DataLoader(test_data,
                                batch_size=BATCH_SIZE,
                                shuffle=False,
                                num_workers=0)

In [18]:
import tqdm

# 학습 횟수
NUM_EPOCHS=8

# loss값 저장
loss_values = []

# 학습 시작
for epoch in tqdm.tqdm(range(NUM_EPOCHS)):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))

    stacked_val_labels = []
    targets_list = []

    print('Training...')

    # train mode 변환
    model.train()
    torch.set_grad_enabled(True)
    total_train_loss = 0
    for i, batch in enumerate(train_dataloader):
        train_status = 'Batch ' + str(i) + ' of ' + str(len(train_dataloader))
        print(train_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(b_input_ids,
                    attention_mask=b_input_mask,
                    labels=b_labels)

        loss = outputs[0]

        total_train_loss = total_train_loss + loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    print('Train loss:' ,total_train_loss)


    # ========================================
    #               Validation
    # ========================================

    print('\nValidation...')

    # evaluation mode로 변환
    model.eval()
    torch.set_grad_enabled(False)
    total_val_loss = 0

    for j, batch in enumerate(val_dataloader):

        val_status = 'Batch ' + str(j) + ' of ' + str(len(val_dataloader))
        print(val_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        outputs = model(b_input_ids,
                attention_mask=b_input_mask,
                labels=b_labels)

        loss = outputs[0]

        total_val_loss = total_val_loss + loss.item()

        preds = outputs[1]
        val_preds = preds.detach().cpu().numpy()
        targets_np = b_labels.to('cpu').numpy()

        targets_list.extend(targets_np)

        if j == 0:  # 첫 번째 batch일 떄
            stacked_val_preds = val_preds

        else:
            stacked_val_preds = np.vstack((stacked_val_preds, val_preds))

    # validation accuracy 계산
    y_true = targets_list
    y_pred = np.argmax(stacked_val_preds, axis=1)

    val_acc = accuracy_score(y_true, y_pred)


    print('Val loss:' ,total_val_loss)
    print('Val acc: ', val_acc)

    torch.save(model.state_dict(), f'/kaggle/working/epoch:{epoch+1}_model.pt')

    # 메모리 관리
    gc.collect()

  0%|          | 0/8 [00:00<?, ?it/s]


======== Epoch 1 / 8 ========
Training...
Train loss: 5836.50705653429

Validation...
Val loss: 479.9613768905401
Val acc:  0.6837967680885136


 12%|█▎        | 1/8 [45:39<5:19:38, 2739.75s/it]


======== Epoch 2 / 8 ========
Training...
Train loss: 4071.668740719557

Validation...
Val loss: 444.2799836695194
Val acc:  0.7073809870432377


 25%|██▌       | 2/8 [1:31:18<4:33:56, 2739.42s/it]

...

 75%|███████▌  | 6/8 [4:33:52<1:31:17, 2738.79s/it]


======== Epoch 7 / 8 ========
Training...
Train loss: 1931.4938318505883

Validation...
Val loss: 490.7500991523266
Val acc:  0.712767506187218


 88%|████████▊ | 7/8 [5:19:30<45:38, 2738.53s/it]  


======== Epoch 8 / 8 ========
Training...
Train loss: 1647.0882160831243

Validation...
Val loss: 517.8110204525292
Val acc:  0.7165526277478527


100%|██████████| 8/8 [6:05:08<00:00, 2738.52s/it]


In [19]:
result_df = pd.DataFrame()

for epoch in range(NUM_EPOCHS):
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, NUM_EPOCHS))
    model.load_state_dict(torch.load(f'/kaggle/working/epoch:{epoch+1}_model.pt'))
    for j, batch in enumerate(test_dataloader):

        inference_status = 'Batch ' + str(j+1) + ' of ' + str(len(test_dataloader))

        print(inference_status, end='\r')

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)


        outputs = model(b_input_ids,
                attention_mask=b_input_mask)

        # 예측값
        preds = outputs[0]
        preds = preds.detach().cpu().numpy()


        if j == 0:
            stacked_preds = preds

        else:
            stacked_preds = np.vstack((stacked_preds, preds))
    preds = np.argmax(stacked_preds, axis=1)
    predicted_genres = [list(label_mapping.keys())[list(label_mapping.values()).index(pred)] for pred in preds]
    result_df[f'label_{epoch+1}'] = predicted_genres

======== Epoch 1 / 8 ========
======== Epoch 2 / 8 ========
======== Epoch 3 / 8 ========
======== Epoch 4 / 8 ========
======== Epoch 5 / 8 ========
======== Epoch 6 / 8 ========
======== Epoch 7 / 8 ========
======== Epoch 8 / 8 ========


In [20]:
test = pd.read_csv("/kaggle/input/jbnu-swuniv-ai/test_data.csv")
result_df['id'] = result_df.index
print(len(result_df))
print(len(test))

29436
29436


In [21]:
result_df

,label_1,label_2,label_3,label_4,label_5,label_6,label_7,label_8,id
0,"Education, Teaching","Education, Teaching",Test Preparation,"Education, Teaching","Science, Math","Education, Teaching","Science, Math","Education, Teaching",0
1,"Arts, Photography","Arts, Photography","Arts, Photography","Arts, Photography","Arts, Photography","Arts, Photography","Arts, Photography","Arts, Photography",1
2,Medical Books,Medical Books,Medical Books,Medical Books,Medical Books,Medical Books,Medical Books,Medical Books,2
3,Reference,Reference,Reference,Reference,Reference,Reference,Reference,Reference,3
4,"Literature, Fiction","Literature, Fiction","Humor, Entertainment",Reference,Reference,Reference,"Humor, Entertainment","Humor, Entertainment",4
...,...,...,...,...,...,...,...,...,...
29431,Test Preparation,Test Preparation,Test Preparation,Test Preparation,Test Preparation,Test Preparation,Test Preparation,Test Preparation,29431
29432,Medical Books,Medical Books,Medical Books,Medical Books,Medical Books,Medical Books,Medical Books,Medical Books,29432
29433,"Sports, Outdoors","Humor, Entertainment","Humor, Entertainment","Humor, Entertainment","Humor, Entertainment","Humor, Entertainment","Humor, Entertainment","Humor, Entertainment",29433
29434,"Religion, Spirituality","Literature, Fiction","Religion, Spirituality","Religion, Spirituality","Literature, Fiction","Literature, Fiction","Literature, Fiction","Literature, Fiction",29434


In [22]:
result_df.to_csv("/kaggle/working/xlmrobertbase_results.csv", index=False)

In [23]:
result_df["label"] = result_df.filter(like='label').apply(lambda x: x.value_counts().idxmax(), axis=1)

In [24]:
result_df[["id", "label"]].to_csv("/kaggle/working/ensemble_xlmrobertbasemodels.csv", index=False)